In [ ]:
# GET LOCATIONS (AND ADDITIONAL PLACE RELATED INFORMATION) FOR ANY NUMBER OF EG COMPANIES BASED ON THE GOOGLE MAPS API

In [ ]:
import json
import requests
import numpy as np
import re
from shapely.geometry import Point, Polygon
import pandas as pd
import urllib
import gspread
import df2gspread as d2g
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

all_company = []


In [ ]:
API_KEY = 'GOOGLE MAPS API'

In [ ]:
companies = ["LIST OF ALL COMPANIES TO QUERY FOR"]

In [ ]:

def place_id(place_id):
    # GET ALL ADDITIONAL INFORMATION BASED ON PLACE ID
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={API_KEY}"

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    result = response.json()['result'] if 'result' in response.json() else None
    
    card = {
        'place_id' : place_id,
        'state': None, 
        'postal_code': None, 
        'locality': None, 
        'website': None, 
        'international_phone_number': None, 
        'name': None
    }
        
    if result:           
        if result['address_components']:
            for i in result['address_components']:
                if 'administrative_area_level_1' in i['types']:
                    card['state'] = i['long_name']
                if 'postal_code' in i['types']:
                    card['postal_code'] = i['long_name']
                if 'locality' in i['types']:
                    card['locality'] = i['long_name']

        card['website'] = result['website'] if 'website' in result else None
        card['international_phone_number'] = result['international_phone_number'] if 'international_phone_number' in result else None
        card['name'] = result['name'] if 'name' in result else None
        
    return card

In [ ]:
def gmaps_search(query):
    # TEXTSEARCH GOOGLE MAPS BASED ON NAME OF COMPANY
    search_endpoint = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

    search_params = {
    'query': query,
    'key': API_KEY,
    'language': 'de'
    }

    response = requests.get( search_endpoint, params=search_params)
    results = response.json()['results']

    all_cards = []
    print(len(results))
    if results:
        for i in range(0, len(results)): 
            card = {}
            

            card['business_status'] = results[i]['business_status'] if 'business_status' in results[i] else np.nan
            card['player_name'] = query
            card['address'] = results[i]['formatted_address'] if 'formatted_address' in results[i] else np.nan
            card['name'] = results[i]['name'] if 'name' in results[i] else np.nan
            card['rating'] = results[i]['rating'] if 'rating' in results[i] else np.nan
            card['user_ratings_total'] = results[i]['user_ratings_total'] if 'user_ratings_total' in results[i] else np.nan

            if 'place_id' in results[i].keys():
                place_card = place_id(results[i]['place_id'])
                card['place_id'] = place_card

            all_cards.append(card)   
    return all_cards



In [ ]:
for c in companies: 
    query_cards = gmaps_search(c)
    all_company.append(query_cards)


In [ ]:
l = []
for comp in [item for sublist in x for item in sublist]:
    d = {
        'business_status' : comp['business_status'],
        'player_name' : comp['player_name'],
        'address' : comp['address'],
        'name' : comp['name'],
        'rating' : comp['rating'],
        'user_ratings_total' : comp['user_ratings_total'],
        'state' : comp['place_id']['state'],
        'postal_code' : comp['place_id']['postal_code'],
        'locality' : comp['place_id']['locality'],
        'website' : comp['place_id']['website'],
        'international_phone_number' : comp['place_id']['international_phone_number']
    }
    
    l.append(d)

In [ ]:
df = pd.DataFrame.from_records(l)

In [ ]:
df.head()

In [ ]:
all_company = [ele for ele in all_company if ele != []]

In [ ]:
company_count = [len(i) for i in all_company]

In [ ]:
# GET LAT AND LNG 

for i, row in df.iterrows():
    address = str(df.at[i,'address'])  
    encoded_address = urllib.parse.quote(address)
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+encoded_address+'&key='+API_KEY)
    resp_json = response.json()
    lat = (resp_json['results'][0]['geometry']['location']['lat'])
    lng = (resp_json['results'][0]['geometry']['location']['lng'])

    df.at[i,'lat'] = lat
    df.at[i,'lng'] = lng

In [ ]:

# LOAD RESULTS TO GOOGLE SHEETS DOCUMENT. YOU NEED TO GET YOUR CREDENTIALS TO DO SO FIRST. PLEASE LOOK THIS STEP UP YOURSELF. 

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'PATH TO jsonFileFromGoogle.json', scope)
gc = gspread.authorize(credentials)

spreadsheet_key = 'GOOGLE SPREADSHEET ID'
wks_name = 'WORKSHEET NAME'
d2g.upload(df.reset_index(), spreadsheet_key, wks_name, credentials=credentials, row_names=True)